In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
songs = pd.read_csv('music.csv')
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [6]:
songs['text'] = songs['text'].str.replace(r'\n', '')

C:\Users\Laksmi Kasetty\AppData\Local\Temp\ipykernel_10324\269676193.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [7]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [8]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [9]:
cosine_similarities = cosine_similarity(lyrics_matrix)

In [10]:
similarities = {}
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [11]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [12]:
recommedations = ContentBasedRecommender(similarities)

In [13]:
recommendation = {
    "song": songs['song'].iloc[20],
    "number_songs": 5 
}
recommedations.recommend(recommendation)

The 5 recommended songs for Don't Be Kind are:
Number 1:
Be Kind To Me by Bread with 0.416 similarity score
--------------------
Number 2:
One Of A Kind by Stevie Wonder with 0.381 similarity score
--------------------
Number 3:
Just One Of A Kind by Hank Snow with 0.38 similarity score
--------------------
Number 4:
This Kind Of Love by Cyndi Lauper with 0.356 similarity score
--------------------
Number 5:
Pure Sin by Carly Simon with 0.283 similarity score
--------------------
